In [2]:
import os
import pandas as pd
import numpy as np

In [ ]:
base_path = os.getcwd()

for fp, pn, fns in os.walk(base_path):
    proj = fp.rsplit("/",1)[1]  
    if (proj.startswith("F")):  
        col = "时间"
    for fn in fns:
        if (fn.endswith("csv")):    
            df = pd.read_csv(  
                os.path.join(fp,fn), 
                encoding="gb2312",
                na_values="-"
            ).fillna(method="pad")  
            if (fn.startswith("20")):  
                of = f"{base_path}/out/{proj}产量.csv"
                cols = ["产量"]
                fmt = "%Y%m%d"
            else:   
                of = f"{base_path}/out/{proj}流量.csv"
                df.rename(columns={"timerstr": "时间"}, inplace=True)   
                cols = ["前段流量计","中段流量计","后段流量计"]
                fmt = "%Y-%m-%d %H:%M:%S"
            head = not os.path.exists(of)   
            
            df[col] = pd.to_datetime(df[col],format=fmt)   
            df = (
                df.set_index(col,drop=True) 
                .sort_index().apply(pd.to_numeric)
            )
            df = df.resample('1D').max()    
            df[cols].to_csv(of,header=head,index=True,mode='a')  
            del df  

In [3]:
base_path = f"{os.getcwd()}/out"
df1 = pd.read_csv(
    os.path.join(base_path,"F405A20产量.csv"),
    index_col="时间",  
    parse_dates=True
)
df2 = pd.read_csv(
    os.path.join(base_path,"F405A20流量.csv"),
    index_col="时间",
    parse_dates=True
)

# df1 = pd.read_csv(
#     os.path.join(base_path,"F426A20产量.csv"),
#     index_col="时间",
#     parse_dates=True
# )
# df2 = pd.read_csv(
#     os.path.join(base_path,"F426A20流量.csv"),
#     index_col="时间",
#     parse_dates=True
# )

In [4]:
df = df1.combine_first(df2) 
产量 = df["产量"]

In [5]:
df = df - df.shift(1,fill_value=0) 
df["产量"] = 产量   
df["气耗/平方米"] = (
    df.filter(regex="流量计$",axis=1).sum(axis=1)   
    .div(df["产量"]).replace([np.inf,-np.inf,np.nan],0) 
)

df.to_csv(f"{base_path}/F405A20单位气耗.csv",encoding="gb2312") 

# df.to_csv(f"{base_path}/F426A20单位气耗.csv",encoding="gb2312")